* オムロン 環境センサ 2JCIE-BU01 のサンプルプログラム
    * オムロン社から提供していただいたサンプルプログラムをベースに作成しています    
* Bluetooth 接続する前のセンサから送信されるアドバタイズ（ブロードキャスト通信）のアドバタイズの時間間隔とデータモードを設定
    * 時間間隔：初期値 100ms (0x00A0) 設定可能範囲 100ms (0x00A0) ～ 10.24s (0x4000)
    * データモード　（1.2.のみ実装）
        1. センサーデータ
        2. 計算値
        3. センサーデータと計算値
        4. センサーデータと計算値のフラグ情報
        5. シリアルナンバー

In [4]:
import sys
import time
import asyncio
from bleak import BleakClient
from struct import pack, unpack
import argparse

# 2JCIE-BU01 address
#ADDRESS = "CD:C7:28:85:D3:AB"

# 2JCIE-BU01 GATT UUIDs
model_number_string_UUID = "00002A24-0000-1000-8000-00805F9B34FB"
mode_change_UUID         = "AB705117-0A3A-11E8-BA89-0ED5F89F718B"
advertise_setting_UUID   = "AB705115-0A3A-11E8-BA89-0ED5F89F718B"
flash_memory_status_UUID = "AB705403-0A3A-11E8-BA89-0ED5F89F718B"

In [13]:
async def waitforwrite(client):
    """
    wait for flash memory write end.
    """
    ret = bytearray([0x01])  # initial
    while ret[0] == 0x01 or ret[0] == 0x04: # loop while memory write/erase.
        time.sleep(0.5)
        ret = await client.read_gatt_char(flash_memory_status_UUID)

    if ret[0] == 0x03:  # flash memory write error
        print("error: mode set error. (flash memory write error)")

async def run(address, adv_mode, interval):
    """
    mode read/set.
    """
    try:
        # open bluetooth client
        async with BleakClient( address ) as client :
            # device check
            ret = await client.read_gatt_char(model_number_string_UUID)
            if ret != b'2JCIE-BU01':
                print("Device with address " + address + " is not 2JCIE-BU01.")
                return
            
            # if no argument, display current mode.
            if adv_mode == 0 :
                print("Address: " + address, "Current advetising interval: ", int.from_bytes(ret[0:2], byteorder='little'), "Current advetising mode: ", ret[2])

	        # check advertise setting (UUID 0x5115)
                ret = await client.read_gatt_char(advertise_setting_UUID)
                if ret[2] == 0x01:
                    print(" advertising mode: 1 [sensor data]")
                elif ret[2] == 0x02:
                    print(" advertising mode: 2 [calcuration data]")
                elif ret[2] == 0x03:
                    print(" advertising mode: 3 [sensor & calcuration data]")
                elif ret[2] == 0x04:
                    print(" advertising mode: 4 [sensor & calcuration flags]")
                elif ret[2] == 0x05:
                    print(" advertising mode: 5 [serial number]")
                elif ret[2] == 0x06:
                    print(" advertising mode: 6 (reserve for future use)")
                elif ret[2] == 0x07:
                    print(" advertising mode: 7 (reserve for future use)")
                elif ret[2] == 0x08:
                    print(" advertising mode: 8 (reserve for future use)")
                else:
                    print(" error: unknown advertise setting " + str(ret[2]))

            else:
	            # read current advertise setting (UUID 0x5115)
                ret = await client.read_gatt_char(advertise_setting_UUID)
                # ret[0],ret[1] are current 'advertising interval'.
                print("Address: " + address, "Current advetising interval: ", int.from_bytes(ret[0:2], byteorder='little'), "Current advetising mode: ", ret[2])

                # set new advertising mode 
                if adv_mode == 1:    # set sensor data mode
                    ret[2] = 0x01
                    print(" set advertising mode: 1 [sensor data]")
                elif adv_mode == 2:  # set calcuration data mode
                    ret[2] = 0x02
                    print(" set advertising mode: 2 [calcuration data]")
                elif adv_mode == 3:  # set sensor & calcuration data mode
                    ret[2] = 0x03
                    print(" set advertising mode: 3 [sensor & calcuration data]")
                elif adv_mode == 4:  # set sensor & calcuration flags mode
                    ret[2] = 0x04
                    print(" set advertising mode: 4 [sensor & calcuration flags]")
                elif adv_mode == 5:  # set serial number mode
                    ret[2] = 0x05
                    print(" set advertising mode: 5 [serial number]")
                else:
                    print( "error: advertising mode needs 1 to 5." )
                    return
                
                if isinstance(interval, int) and int.from_bytes(ret[0:2], byteorder='little') != interval and 100 <= interval <= 10240:
                    print(f" set advertising interval: {interval} ms")
                    b_interval = pack('<H', interval)
                    ret[0] = b_interval[0]
                    ret[1] = b_interval[1]
                    
                await client.write_gatt_char(advertise_setting_UUID, ret)
                await waitforwrite( client )

    except Exception as e:
        # bluetooth client can not open
        print("Device with address " + address + " was not found.")
        print(e)
        sys.exit()

## ここを変更し、アドバタイジングモードの確認・設定を行う
address = 'F9:E9:A4:FB:96:C9' # 2JCIE-BU01 address
adv_mode = 2 # アドバタイジングのモード 1: sensor data mode 2: calcuration data mode 3: sensor & calcuration data mode 4: sensor & calcuration flags mode 5: serial number mode 0: display current mode
interval = 1000 # アドバタイジングの間隔 100ms - 10240ms

await run(address, adv_mode, interval)

Address: F9:E9:A4:FB:96:C9 Current advetising interval:  1000 Current advetising mode:  1
 set advertising mode: 2 [calcuration data]
